In [ ]:
!git clone https://github.com/openai/shap-e

Cloning into 'shap-e'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 336 (delta 35), reused 13 (delta 13), pack-reused 281 (from 2)
Receiving objects: 100% (336/336), 11.72 MiB | 8.95 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [ ]:
%cd shap-e



/content/shap-e


In [ ]:
!pip install -e .

Obtaining file:///content/shap-e
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-install-mlj1zefy/clip_f67026e651e84b3a9559abee86521186
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-install-mlj1zefy/clip_f67026e651e84b3a9559abee86521186
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.9 MB/s eta 0:00:00
   

In [ ]:
# Import libraries
import torch
from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget, decode_latent_mesh
from IPython.display import display
import os
from tqdm import tqdm
import time


In [ ]:


# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Using device: {device}")

# Load models
print("📦 Loading models...")
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

# Create output directory
os.makedirs("shap_e_outputs", exist_ok=True)

# List of UAV prompts
aircraft_prompts = [
    "/content/shap-e/samples/a_spaceship/1.gif"
]
# Settings
batch_size = 1
guidance_scale = 20.0
render_mode = 'nerf'
size = 256  # Higher camera size for better quality
karras_steps = 256  # More diffusion steps for better model
cameras = create_pan_cameras(size, device)

# Timer start
start_time = time.time()

# Generate models
for i, prompt in enumerate(aircraft_prompts):
    print(f"🚀 Generating: {prompt}")

    latents = sample_latents(
        batch_size=batch_size,
        model=model,
        diffusion=diffusion,
        guidance_scale=guidance_scale,
        model_kwargs=dict(texts=[prompt] * batch_size),
        progress=True,    # progress bar
        clip_denoised=True,
        use_fp16=True,
        use_karras=True,
        karras_steps=karras_steps,
        sigma_min=1e-3,
        sigma_max=160,
        s_churn=0,
    )

    for j, latent in enumerate(latents):
        # Render preview
        images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
        display(gif_widget(images))

        # Clean filename
        name = prompt.lower().replace(" ", "_").replace("/", "_")

        # Save mesh
        mesh = decode_latent_mesh(xm, latent).tri_mesh()
        with open(f'shap_e_outputs/{name}.ply', 'wb') as f:
            mesh.write_ply(f)
        with open(f'shap_e_outputs/{name}.obj', 'w') as f:
            mesh.write_obj(f)

# Timer end
end_time = time.time()
elapsed_time = end_time - start_time
minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)
print(f"✅ All models generated and saved in /shap_e_outputs folder")
print(f"🕰️ Total time taken: {minutes} minutes {seconds} seconds")


✅ Using device: cuda
📦 Loading models...


/content/shap-e/shap_e/models/nn/checkpoint.py:31: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/shap-e/shap_e/models/nn/checkpoint.py:43: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/content/shap-e/shap_e/models/nn/checkpoint.py:61: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/content/shap-e/shap_e/models/nn/checkpoint.py:86: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


  0%|          | 0.00/1.78G [00:00<?, ?iB/s]

100%|███████████████████████████████████████| 890M/890M [00:22<00:00, 41.6MiB/s]


  0%|          | 0.00/1.26G [00:00<?, ?iB/s]

🚀 Generating: /content/shap-e/samples/a_spaceship/1.gif


  0%|          | 0/256 [00:00<?, ?it/s]

HTML(value='<img src="data:image/gif;base64,R0lGODlhAAEAAYcAAADV3QDU3ADT3ADS3ADS2wDR2wDQ2wDR2QDP2gDO2gDN2gDN1g…

/content/shap-e/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/content/shap-e/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


✅ All models generated and saved in /shap_e_outputs folder
🕰️ Total time taken: 4 minutes 23 seconds
